In [47]:
# 数据库连接
import psycopg2
import copy
map_name="penn"
conn = psycopg2.connect(database=map_name, user="postgres", password="1997pgsql", host="127.0.0.1", port="5432")
print("Connect database successfully") 
cur=conn.cursor()

Connect database successfully


In [48]:
conn.rollback()

# 选出作为POI的node,新建一张表来保存

In [49]:
%%time
cur.execute('drop table if exists qualified_poi')
cur.execute('create table qualified_poi (node_id bigint not null,k text,category_id int)')
conn.commit()
category_sql_list=["k = 'power'","k ='name'","k ='shop'","k ='leisure'","k = 'religion'","k like 'service%'",
                  "k ='vending'","k = 'building'","k='beer'","k='restaurant' ","k like 'drink%' ","k like 'toilet%'","k like 'marketplace%'",
                   " k like 'cuisine%'","k ='brewery'","k='consulate'","k='sauna'","k='sculptor'","k='distillery'","k='furniture'",
                  "k='pastry'","k like 'theatre%'","k='university'","k='tower'","k='military'","k='store'","k='clothes'","k='lottery'",
                  "k='breakfast'","k='office' ","k='grave'"," k='parking'","k='police'","k='dinner'","k='golf'","k='preschool'","k='church'",
                  "k='coffee'"," k='atm'","k='massage'","k='museum'","k='bakery'","k='park'","k='hospital'","k='jewelry'",
                  "k='goods'","k='farm'","k='artwork' ","k='amenity'","k='healthcare' ","k='karaoke'","k='cafe'","k='brand'"]
insert_poi_sql="""
insert into qualified_poi
select node_id,k ,0
from node_tags 
where %s
"""
select_poi_sql="""
select node_id,k 
from node_tags 
where %s
"""
set_category_id_sql = """
update qualified_poi set category_id=%d where %s
"""
category_count=0
index=0
for i in range(len(category_sql_list)):
    try:
        category_string=category_sql_list[i]
        cur.execute(select_poi_sql%(category_string))
        result=cur.fetchall()
        if len(result)==0:
            print(category_string)
            continue
        cur.execute(insert_poi_sql%(category_string))
        cur.execute(set_category_id_sql%(index,category_string))
        conn.commit()
        print('category %d done.'%(index))
        index+=1
    except Exception as e:
        print(e)
        conn.rollback()
print('extract qualified poi done.')

category 0 done.
category 1 done.
category 2 done.
category 3 done.
category 4 done.
category 5 done.
category 6 done.
category 7 done.
category 8 done.
category 9 done.
category 10 done.
category 11 done.
k like 'marketplace%'
category 12 done.
category 13 done.
category 14 done.
k='sauna'
k='sculptor'
category 15 done.
category 16 done.
k='pastry'
category 17 done.
category 18 done.
category 19 done.
category 20 done.
k='store'
category 21 done.
k='lottery'
category 22 done.
category 23 done.
k='grave'
category 24 done.
category 25 done.
category 26 done.
category 27 done.
category 28 done.
category 29 done.
category 30 done.
category 31 done.
category 32 done.
category 33 done.
k='bakery'
category 34 done.
k='hospital'
k='jewelry'
k='goods'
k='farm'
k='artwork' 
category 35 done.
category 36 done.
k='karaoke'
k='cafe'
category 37 done.
extract qualified poi done.
Wall time: 11.5 s


# 获取每个category的POI总数

In [50]:
sql_str="select category_id, count(distinct node_id) as num from qualified_poi group by category_id"
cur.execute(sql_str)
result=cur.fetchall()
category_num=[]
category_count=len(result)
for item in result:
    category_num.append(item[1])

In [51]:
print(category_num)

[61253, 82028, 15320, 3939, 5070, 173, 108, 20570, 2, 5, 95, 246, 4992, 28, 1, 1, 2, 10, 2, 4, 38, 260, 3, 2910, 397, 2, 1, 287, 9, 1, 1, 282, 1, 16, 2, 42410, 1833, 7426]


# 从foursquare获取评分

In [20]:
foursquareRating=[]
with open('../foursquare/foursquareRating.txt','r') as f:
    for line in f:
        data=line.split()
        for item in data:
            foursquareRating.append(item)

print(len(foursquareRating))

84291


# 随机为node赋值为POI

In [53]:
import random

chosen=set()
cur.execute("select count(distinct id) from qualified_nodes")
node_len=cur.fetchall()[0][0]

with open('../../map_'+map_name+'/category.txt','w') as f:
    for i in range(len(category_num)):
        num=category_num[i]
        while num>=0:
            choose_node=random.randint(0,node_len-1)
            while choose_node in chosen:
                choose_node=random.randint(0,node_len-1)
            chosen.add(choose_node)
            num-=1
            f.write(str(choose_node)+' '+str(i)+' '+str(random.choice(foursquareRating))+'\n')

# 读取edge.txt文件，为路网添加POI

### 这一步主要是为Real Datasets for Spatial Databases的四个没有POI的数据集构造POI数据

In [31]:
# 对于edge文件，需要将它的edge ID去掉
map_name='TG'
edge_file='../RDSD/'+map_name+'/edgeo.txt'
lonlat_file='../RDSD/'+map_name+'/lonlat.txt'

output_edge_file='../RDSD/'+map_name+'/edge.txt'
output_edge=open(output_edge_file,'w')
category_file='../RDSD/'+map_name+'/category.txt'

with open(edge_file,'r') as input_edge:
    for line in input_edge:
        data=line.split(' ')
        output_edge.write(data[1]+' '+data[2]+' '+data[3])
output_edge.close()

In [32]:
# 对于lonlat文件，读取所有node的ID
node_set=set()
with open(lonlat_file,'r') as input_lonlat:
    for line in input_lonlat:
        data=line.split(' ')
        node_set.add(int(data[0]))
node_num=len(node_set)
print(node_num)

18263


In [33]:
import random

POI_num=int(node_num*0.13)
category_num=random.randint(0,100)
category_list=[random.randint(10,200) for i in range(category_num)]
# 添加POI并构建category.txt
# category的文件格式 node_id category_id rating
chosen=set()
with open(category_file,'w') as output_category:
    for i in range(category_num):
        c_num=category_list[i]
        while c_num>=0:
            choose_node=random.randint(0,node_num-1)
            while choose_node in chosen:
                choose_node=random.randint(0,node_num-1)
            chosen.add(choose_node)
            c_num-=1
            output_category.write(str(choose_node)+' '+str(i)+' '+str(random.choice(foursquareRating))+'\n')